In [1]:
import pandas as pd
from pandasql import sqldf
import sqlalchemy

In [2]:
cd

C:\Users\mojde


In [39]:
prices = pd.read_csv('prices.csv')
prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [4]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [40]:
# Replace "dollars" with blan sapace and then, Convert the 'price' column to a numerical data type and
prices['price'] = pd.to_numeric(prices['price'].str.replace(' dollars', ''), errors='coerce')

In [53]:
#check if there's any rows conating the word dollar
prices.loc[prices['price'] == 'dollars']

,listing_id,price,nbhood_full


In [47]:
prices.astype({'price': 'int32'}).dtypes

listing_id      int64
price           int32
nbhood_full    object
dtype: object

In [8]:
reviews = pd.read_csv('reviews.csv')
reviews.head()

,Unnamed: 0,listing_id,host_name,last_review,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,booked_days_365
0,0,2595,Jennifer,2019-05-21 00:00:00,1,45,0.38,2,355,10
1,1,3831,LisaRoxanne,2019-07-05 00:00:00,1,270,4.64,1,194,171
2,2,5099,Chris,2019-06-22 00:00:00,3,74,0.59,1,129,236
3,3,5178,Shunichi,2019-06-24 00:00:00,2,430,3.47,1,220,145
4,4,5238,Ben,2019-06-09 00:00:00,1,160,1.33,4,188,177


In [9]:
reviews.astype({'number_of_reviews': 'int32'}).dtypes

Unnamed: 0                          int64
listing_id                          int64
host_name                          object
last_review                        object
minimum_nights                      int64
number_of_reviews                   int32
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
booked_days_365                     int64
dtype: object

In [10]:
room_types = pd.read_csv('room_types.csv')
room_types.head()

,Unnamed: 0,listing_id,description,room_type
0,0,2595,Skylit Midtown Castle,entire home/apt
1,1,3831,Cozy Entire Floor of Brownstone,entire home/apt
2,2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,3,5178,Large Furnished Room Near B'way,private room
4,4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt


# First approach
Use Python corr() function

In [11]:


import pandas as pd

# Assuming you have two DataFrames: prices and reviews

# Calculate correlations
correlation_number_of_reviews = prices['price'].corr(reviews['number_of_reviews'])
correlation_reviews_per_month = prices['price'].corr(reviews['reviews_per_month'])

# Print the results
print("Correlation (number_of_reviews):", correlation_number_of_reviews)
print("Correlation (reviews_per_month):", correlation_reviews_per_month)

Correlation (number_of_reviews): -0.06046667472746895
Correlation (reviews_per_month): -0.0497316790187169


# Second approach
Using SQL querying directly in Python environment



In [66]:
query = '''
SELECT CORR(price, number_of_reviews) AS correlation_number_of_reviews, CORR(price, reviews_per_month) AS correlation_reviews_per_month
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
'''

In [69]:
result = sqldf(query)
result

# OperationalError: no such function: CORR

PandaSQLException: (sqlite3.OperationalError) no such function: CORR
[SQL: 
SELECT CORR(price, number_of_reviews) AS correlation_number_of_reviews, CORR(price, reviews_per_month) AS correlation_reviews_per_month
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Third approach
Using our memory as a database. This is however unneccary since it's pretty much like the second step.

In [74]:
import pandas as pd
from sqlalchemy import create_engine

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:', echo=False)

# Create the prices and reviews dataframes
prices_df = pd.read_csv('prices.csv')
reviews_df = pd.read_csv('reviews.csv')

# Insert data into prices table
prices_df.to_sql('prices', con=engine, if_exists='replace', index=False)

# Insert data into reviews table
reviews_df.to_sql('reviews', con=engine, if_exists='replace', index=False)

# Perform the SQL query with CORR function
query = '''
SELECT 
    CORR(price, number_of_reviews) AS correlation_number_of_reviews,
    CORR(price, reviews_per_month) AS correlation_reviews_per_month
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
'''
result = pd.read_sql_query(query, con=engine)

# Close the database connection
engine.dispose()

# Print the result
print(result)


OperationalError: (sqlite3.OperationalError) no such function: CORR
[SQL: 
SELECT 
    CORR(price, number_of_reviews) AS correlation_number_of_reviews,
    CORR(price, reviews_per_month) AS correlation_reviews_per_month
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [75]:
 # OperationalError: no such function: CORR

# The fourth approach 
Creating a pandasql object. This one might finally work but fixning the errors while inserting the datas in the object won't worth it, I guess.

In [55]:
import pandas as pd
from pandasql import sqldf

# Create the prices and reviews dataframes
prices_df = pd.read_csv('prices.csv')
reviews_df = pd.read_csv('reviews.csv')

# Remove "dollars" symbol from price values in prices_df
prices_df['price'] = prices_df['price'].str.replace(' dollars', '').astype(float)

# Define the custom environment with the math module
env = {'math': math}

# Create the pandasql object
pysqldf = lambda q: sqldf(q, env=env)

# Create the prices and reviews tables
pysqldf("CREATE TABLE prices (listing_id INT, price FLOAT, nbhood_full TEXT);")
pysqldf("CREATE TABLE reviews (listing_id INT, number_of_reviews INT, reviews_per_month FLOAT);")

# Convert prices_df and reviews_df to string representation
prices_values = ",".join(prices_df.astype(str).apply(lambda x: "(" + ", ".join(x) + ")", axis=1))
reviews_values = ",".join(reviews_df.astype(str).apply(lambda x: "(" + ", ".join(x) + ")", axis=1))

# Insert data into prices and reviews tables
pysqldf("INSERT INTO prices VALUES " + prices_values + ";")
pysqldf("INSERT INTO reviews VALUES " + reviews_values + ";")

# Perform the SQL query with AVG and STD functions
query = '''
SELECT AVG(price * number_of_reviews) / (STD(price) * STD(number_of_reviews)) AS correlation_number_of_reviews,
       AVG(price * reviews_per_month) / (STD(price) * STD(reviews_per_month)) AS correlation_reviews_per_month
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
'''
result = pysqldf(query)

# Print the result
print(result)


PandaSQLException: (sqlite3.OperationalError) near "Hill": syntax error
[SQL: INSERT INTO prices VALUES (2595, 225.0, Manhattan, Midtown),(3831, 89.0, Brooklyn, Clinton Hill),(5099, 200.0, Manhattan, Murray Hill),(5178, 79.0, Manhattan, Hell's Kitchen),(5238, 150.0, Manhattan, Chinatown),(5295, 135.0, Manhattan, Upper West Side),(5441, 85.0, Manhattan, Hell's Kitchen),(5803, 89.0, Brooklyn, South Slope),(6021, 85.0, Manhattan, Upper West Side),(6848, 140.0, Brooklyn, Williamsburg),(7097, 215.0, Brooklyn, Fort Greene),(7322, 140.0, Manhattan, Chelsea),(7726, 99.0, Brooklyn, Crown Heights),(8024, 130.0, Brooklyn, Park Slope),(8025, 80.0, Brooklyn, Park Slope),(8110, 110.0, Brooklyn, Park Slope),(8490, 120.0, Brooklyn, Bedford-Stuyvesant),(8505, 60.0, Brooklyn, Windsor Terrace),(9518, 44.0, Manhattan, Inwood),(9657, 180.0, Manhattan, East Village),(9668, 50.0, Manhattan, Harlem),(9704, 52.0, Manhattan, Harlem),(9782, 55.0, Brooklyn, Greenpoint),(9783, 50.0, Manhattan, Harlem),(10452, 70.0

# Fifth Approach

Could be using CORR() formula but you will still face challenges when calculating the SQRT() as it won't be recognzed.

## To make sure that our third approach's syntax is correct you could go one Run these cells. I replace AVG with COR()

In [79]:
#using sqlite3 

import pandas as pd
import sqlite3
import math

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Create the prices and reviews dataframes
prices_df = pd.read_csv('prices.csv')
reviews_df = pd.read_csv('reviews.csv')

# Insert data into prices table
prices_df.to_sql('prices', conn, if_exists='replace', index=False)

# Insert data into reviews table
reviews_df.to_sql('reviews', conn, if_exists='replace', index=False)

# Perform the SQL query with AVG and manual standard deviation calculation
query = '''
SELECT 
    AVG(price * number_of_reviews),
    AVG(price * reviews_per_month)
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
'''
result = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Print the result
print(result)


   AVG(price * number_of_reviews)  AVG(price * reviews_per_month)
0                     5208.623825                      266.718915


In [80]:
#using sqlalchemy
import pandas as pd
from sqlalchemy import create_engine

# Create an in-memory SQLite database
engine = create_engine('sqlite:///:memory:', echo=False)

# Create the prices and reviews dataframes
prices_df = pd.read_csv('prices.csv')
reviews_df = pd.read_csv('reviews.csv')

# Insert data into prices table
prices_df.to_sql('prices', con=engine, if_exists='replace', index=False)

# Insert data into reviews table
reviews_df.to_sql('reviews', con=engine, if_exists='replace', index=False)

# Perform the SQL query with AVG and CORR functions
query = '''
SELECT 
    AVG(price * number_of_reviews),
    AVG(price * reviews_per_month)
FROM prices
JOIN reviews ON prices.listing_id = reviews.listing_id;
'''
result = pd.read_sql_query(query, con=engine)

# Close the database connection
engine.dispose()

# Print the result
print(result)


   AVG(price * number_of_reviews)  AVG(price * reviews_per_month)
0                     5208.623825                      266.718915
